# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [346]:
# names us
from glob import glob

def df_one_year(file):
    columns = ['name', 'gender', 'births']
    
    df = pd.DataFrame([])
    df = df.append(pd.read_csv(file, names=columns))
    df.insert(0, 'year', int(file[len('names/yob'):-len('.txt')]))
    
    return df

def df_names_us():
    df = pd.DataFrame([])
    
    # liste de tous les fichiers .txt dans le dossier names
    birth_files = sorted(glob('names/yob*.txt'))
    
    df = pd.concat((df_one_year(file) for file in birth_files), axis=0)
    
    # Indexer le DataFrame
    df = df.reset_index(drop=True)
    
    return df

In [348]:
df_names_us().head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [349]:
# names fr
def df_names_fr():
    df0 = pd.DataFrame()
    df0 = pd.read_csv('nat2020.csv', delimiter=';')
    
    df = df0.copy()
    
    # rennomer les colonnes
    df.rename(columns={'sexe': 'gender', 'preusuel': 'name', 'annais': 'year', 'nombre': 'births'}, inplace=True)
    
    # re-ordonner les colonnes
    df = df.reindex(columns=['year', 'name', 'gender', 'births'])
    
    # traitement du gender
    mapping = {1 : 'M', 2 : 'F'}
    df['gender'] = df['gender'].map(mapping)
    
    # dtype année : object --> float()
    # downcast='integer' ==> si year = 'xxxx', then year = NaN
    df['year'] = pd.to_numeric(df['year'], errors='coerce', downcast='integer')
    
    # Vérification du type de value pour la colonne 'name'
    #df['name'].apply(type).value_counts() # On observe 2 données de type non str
    # Rechercher les 2 données différentes dans la colonne
    #df.loc[df['name'].apply(lambda x: not isinstance(x, str))]
    # Modification de ces données : des personnes s'apellent NA dans la base de données.
    df.loc[df['name'].apply(lambda x: not isinstance(x, str)) == True, 'name'] = 'NA'
    
    # suppression des 'xxxx' dans l'année
    df = df.dropna(axis=0, how='any')
    
    # dtype année : object --> int()
    df['year'] = df['year'].apply(lambda x: int(x))
    
    # suppresion des prenoms dont la len est inf à 2
    df = df[df['name'].astype(str).apply(len) >= 2]
    
    # Modification des names : 1ere lettre en MAJ puis le reste en min
    df['name'] = df['name'].str.capitalize()
    
    # suppresion de _prenoms_rare
    df = df[df['name'].astype(str) != '_prenoms_rares']
    
    # Trier les données
    df = df.sort_values(by=['year','gender','births','name'], axis=0, ascending=[True, True, False, True])
    
    # Indexer le DataFrame
    df = df.reset_index(drop=True)
    
    return df

In [350]:
df1 = df_names_fr()
df1.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [363]:
# taux de change
def df_taux_change(devises):
    # Lecture du fichier csv
    df1 = pd.read_csv('Webstat_Export_20210929.csv', delimiter=';', header=None)#, parse_dates=True, infer_datetime_format=True)
    
    # Modification des noms des colonnes (ex: Franc suisse (CHF) --> CHF ) 
    df1.columns = df1.iloc[0].apply(lambda x: x[-4:-1])
    
    # Création DataFrame avec les devises que l'on souhaite
    df = pd.DataFrame([], columns=devises)
    df = df1[devises].copy()
    
    # Indexation
    df.set_index(pd.to_datetime(df1['re '], errors='coerce', format='%d/%m/%Y'), inplace=True)
    #df.rename_axis('Dates')
    
    # to_numeric + NAN
    df = df.applymap(lambda x: str(x).replace(',', '.'))
    df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
    
    # Suppression des NAN
    df = df.dropna(axis=0, how='any')
    
    df.set_index(pd.to_datetime(df.index, format='%d/%m/%Y'), inplace=True)
    
    return df.rename_axis(None)

In [364]:
df2 = df_taux_change(['CHF', 'GBP', 'USD'])
df2.head()

,CHF,GBP,USD
2021-09-28,1.0840,0.85950,1.1678
2021-09-27,1.0850,0.85420,1.1698
2021-09-24,1.0830,0.85703,1.1719
2021-09-23,1.0831,0.85495,1.1715
2021-09-22,1.0820,0.86000,1.1729


### Tests

In [341]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [342]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [343]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.948s

OK
